# Tâche de Classification des recettes par le type de plat
Auteurs :     
  - BOURDERE ANDREOU Nathan
  - CHUPIN Yannis

## IMPORTS

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import seaborn as sns
!pip install gensim
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
!pip install nltk==3.8.1
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.manifold import TSNE
nltk.download('stopwords')
nltk.download('punkt')
from sklearn.metrics import confusion_matrix
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 12.0 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.6 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.9.1
    Uninstalling nltk-3.9.1:
      Successfully uninstalled nltk-3.9.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
textblob 0.19.0 requires nltk>=3.9, but you have nltk 3.8.1 which is incompatible.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Pre-processing

### Récupération des données de train et de test

In [3]:
train_data = pd.read_csv('./sample_data/train.csv',encoding="utf-8")
test_data = pd.read_csv('./sample_data/test.csv')
stemmer = SnowballStemmer('french')
stopwords = set(stopwords.words('french'))
print(train_data.shape,test_data.shape)
train_data.head()

(12473, 7) (1388, 7)


,doc_id,titre,type,difficulte,cout,ingredients,recette
0,recette_221358.xml,"Feuilleté de saumon et de poireau, sauce aux c...",Plat principal,Facile,Moyen,- 1 gros pavé de saumon - 100 g de crevettes d...,Couper finement le blanc et un peu de vert des...
1,recette_48656.xml,Cake poulet/moutarde/amandes,Entrée,Très facile,Bon marché,- 3 œufs - 150 g de farine - 1 sachet de levur...,"Couper finement l'échalote, la faire revenir à..."
2,recette_30049.xml,Bûche à la truite fumée (7ème rencontre),Entrée,Moyennement difficile,Assez Cher,- 800 g de filet de truite saumonnée fumée en ...,Faites blanchir les épinards à l'eau bouillant...
3,recette_71424.xml,Gâteau au yaourt au coco sans huile de laetitia,Dessert,Très facile,Bon marché,- 1 pot de yaourt - 1 pot de lait de coco - 3 ...,Mélanger dans l'ordre tous les ingrédients en ...
4,recette_217204.xml,Crêpes au canard laqué,Entrée,Moyennement difficile,Moyen,- 90 g de farine - 45 g de maïzena - 2 œufs - ...,"Fouetter les œufs avec l'eau, le lait et le su..."


In [6]:
print(len(train_data[train_data['type']== "Plat principal"] ), len(train_data[train_data['type']== "Dessert"]), len(train_data[train_data['type']== "Entrée"]))

5802 3762 2909


In [8]:
nb_1, nb_b, nb_c = len(test_data[test_data['type']== "Plat principal"] ), len(test_data[test_data['type']== "Dessert"]), len(train_data[train_data['type']== "Entrée"])
print(nb_1, nb_b, nb_c)

644 407 2909


In [7]:
print(5802 / (5802+3762+2909), 3762 / (5802+3762+2909), 2909 / (5802+3762+2909))

0.465164755872685 0.3016114807985248 0.2332237633287902


In [ ]:
print(nb_1/(nb_1+ nb_b+ nb_c), nb_b/(nb_1+ nb_b+ nb_c), nb_c/(nb_1+ nb_b+ nb_c))

### Gestion des valeurs manquantes, concaténation des caractéristiques clés de texte et Pre-processing simple des textes combinés


In [ ]:
train_data.dropna(axis=0,inplace=True)
test_data.dropna(axis=0,inplace=True)
text_data = pd.concat([train_data['ingredients'],train_data['recette'],train_data['titre']])
sentences = [simple_preprocess(text) for text in text_data]
data = pd.concat([train_data, test_data])

## 1ère Approche de Pre-processing : CamemBERT

### Installation de torch nécéssaire

In [ ]:
!pip install torch


### Imports nécéssaires à l'utilisation de Camembert

In [ ]:
from transformers import CamembertTokenizer, CamembertModel
import torch

### Utilisation du GPU, instantiation du modèle CamemBERT et fonctions de tokenization et vectorization des embeddings de chaque texte ainsi que de combinaison de chaque vecteur sans pondération

In [ ]:
# Vérifier si un GPU est disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# Charger le tokenizer et le modèle sur le GPU-
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')
model = CamembertModel.from_pretrained('camembert-base').to(device)

def get_text_vector_bert(text, model, tokenizer):
    # Tokenization et déplacement sur GPU
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512).to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    embeddings = outputs.last_hidden_state
    text_vector = torch.mean(embeddings, dim=1).squeeze()

    return text_vector.cpu().numpy()  # Déplacement vers CPU avant conversion en numpy

def get_combined_vector_bert(ingredients, recette, titre, model, tokenizer):
    # Obtenir les vecteurs pour chaque champ de texte
    vector_ingredients = get_text_vector_bert(ingredients, model, tokenizer)
    vector_recette = get_text_vector_bert(recette, model, tokenizer)
    vector_titre = get_text_vector_bert(titre, model, tokenizer)

    # Combiner les vecteurs (ici par addition)
    combined_vector = vector_ingredients + vector_recette + vector_titre

    return combined_vector


cuda


### Application des fonctions aux données, Drop des caractéristiques non-clés et création d'une caractéristique comprenant les embeddings de mots

In [ ]:
train_data_b= train_data.copy()
test_data_b = test_data.copy()

train_data_b['vector'] = train_data_b.apply(lambda row: get_combined_vector_bert(row['ingredients'], row['recette'], row['titre'], model, tokenizer), axis=1)
test_data_b['vector'] = test_data_b.apply(lambda row: get_combined_vector_bert(row['ingredients'], row['recette'], row['titre'], model, tokenizer), axis=1)

train_data_b = train_data_b.drop(['ingredients', 'recette', 'titre', 'difficulte', 'cout', 'doc_id'], axis=1)
test_data_b = test_data_b.drop(['ingredients', 'recette', 'titre', 'difficulte', 'cout', 'doc_id'], axis=1)

In [ ]:
train_data_b,val_data_b = train_test_split(train_data_b, test_size=0.2, random_state=42)

In [ ]:
X_train_b = np.array(train_data_b['vector'].tolist())
y_train_b = np.array(train_data_b['type'].tolist())
X_val_b = np.array(val_data_b['vector'].tolist())
y_val_b = np.array(val_data_b['type'].tolist())
X_test_b = np.array(test_data_b['vector'].tolist())
y_test_b = np.array(test_data_b['type'].tolist())

In [ ]:
# Entraîner le modèle SVC
svc_model_b = SVC(kernel='linear')
svc_model_b.fit(X_train_b, y_train_b)

# Entraîner le modèle Naive Bayes
nb_model_b = GaussianNB()
nb_model_b.fit(X_train_b, y_train_b)

# Entraîner le modèle Random Forest
rf_model_b = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_b.fit(X_train_b, y_train_b)


RandomForestClassifier(random_state=42)

## 2ème Approche de Pre-processing : Word2Vec

In [ ]:
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, workers=4)

### Fonctions de génération des embeddings de mots puis combinaison de chaque caractéristique sans pondération

In [ ]:
def get_text_vector(text, model):
       tokens = simple_preprocess(text)
       vectors = [model.wv[token] for token in tokens if token in model.wv]
       if vectors:
           return np.mean(vectors, axis=0)
       else:
           return np.zeros(model.vector_size)  # Vecteur nul si aucun mot n'est trouvé

def get_combined_vector(ingredients, recette, titre, model):
       ing_vector = get_text_vector(ingredients, model)
       recipe_vector = get_text_vector(recette, model)
       title_vector = get_text_vector(titre, model)

       return ing_vector + recipe_vector + title_vector

In [ ]:
train_data_w= train_data.copy()
test_data_w = test_data.copy()

train_data_w['vector'] = train_data_w.apply(lambda row: get_combined_vector(row['ingredients'], row['recette'], row['titre'], word2vec_model), axis=1)
test_data_w['vector'] = test_data_w.apply(lambda row: get_combined_vector(row['ingredients'], row['recette'], row['titre'], word2vec_model), axis=1)


train_data_w = train_data_w.drop(['ingredients', 'recette', 'titre','difficulte','cout','doc_id'], axis=1)
test_data_w = test_data_w.drop(['ingredients', 'recette', 'titre','difficulte','cout','doc_id'], axis=1)

train_data_w

,type,vector
0,Plat principal,"[1.8405869, 0.32682198, 0.06141232, 0.7335325,..."
1,Entrée,"[0.7830906, 0.74097097, 0.76809365, -1.127624,..."
2,Entrée,"[1.1371391, 0.86213243, 0.36356083, 0.30530527..."
3,Dessert,"[1.2620687, 1.2913873, -0.35943377, -1.4883401..."
4,Entrée,"[1.0761926, 0.16296399, -0.8636645, -0.9163776..."
...,...,...
12468,Plat principal,"[1.158174, -0.3817521, 0.809083, -0.61698663, ..."
12469,Plat principal,"[1.7130607, -0.5851704, -0.51417977, 0.3567018..."
12470,Plat principal,"[0.6671086, 0.48680797, -0.46964008, -0.394424..."
12471,Dessert,"[2.010375, 0.760108, -0.022740632, -1.6133845,..."


In [ ]:
train_data_w, val_data_w = train_test_split(train_data_w, test_size=0.2, random_state=42)

In [ ]:
X_train_w = np.array(train_data_w['vector'].tolist())
y_train_w = np.array(train_data_w['type'].tolist())
X_val_w = np.array(val_data_w['vector'].tolist())
y_val_w = np.array(val_data_w['type'].tolist())
X_test_w = np.array(test_data_w['vector'].tolist())
y_test_w = np.array(test_data_w['type'].tolist())

In [ ]:
# Entraîner le modèle SVC
svc_model_w = SVC(kernel='linear')
svc_model_w.fit(X_train_w, y_train_w)

# Entraîner le modèle Naive Bayes
nb_model_w = GaussianNB()
nb_model_w.fit(X_train_w, y_train_w)

# Entraîner le modèle Random Forest
rf_model_w = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_w.fit(X_train_w, y_train_w)


RandomForestClassifier(random_state=42)

In [ ]:
train_data_tf= train_data.copy()
test_data_tf = test_data.copy()

 # Création du vectoriseur
vectorizer = TfidfVectorizer()

# Combinaison des textes d'entraînement pour l'ajustement du vectoriseur
train_combined_texts = train_data_tf['ingredients'] + " " + train_data_tf['recette'] + " " + train_data_tf['titre']
vectorizer.fit(train_combined_texts)

test_combined_texts = test_data_tf['ingredients'] + " " + test_data_tf['recette'] + " " + test_data_tf['titre']

# Transformation plus efficace (tout d'un coup plutôt que ligne par ligne)
train_vectors = vectorizer.transform(train_combined_texts).toarray()
test_vectors = vectorizer.transform(test_combined_texts).toarray()

# Ajout des vecteurs aux DataFrames
train_data_tf['vector'] = list(train_vectors)
test_data_tf['vector'] = list(test_vectors)

# Suppression des colonnes inutiles
train_data_tf = train_data_tf.drop(['ingredients', 'recette', 'titre', 'difficulte', 'cout', 'doc_id'], axis=1)
test_data_tf = test_data_tf.drop(['ingredients', 'recette', 'titre', 'difficulte', 'cout', 'doc_id'], axis=1)

NameError: name 'get_combined_vector' is not defined

In [ ]:
train_data_tf, val_data_tf = train_test_split(train_data_tf, test_size=0.2, random_state=42)

(9978, 2) (2495, 2) (1388, 2)


In [ ]:
def preprocess_text(text):
  tokens = nltk.word_tokenize(text)
  tokens = [stemmer.stem(token.lower()) if token.lower() not in stopwords else token.lower() for token in tokens]
  return "".join(tokens)

In [ ]:
X_train_tf = np.array(train_data_tf['vector'].tolist())
y_train_tf = np.array(train_data_tf['type'].tolist())
X_val_tf = np.array(val_data_tf['vector'].tolist())
y_val_tf = np.array(val_data_tf['type'].tolist())
X_test_tf = np.array(test_data_tf['vector'].tolist())
y_test_tf = np.array(test_data_tf['type'].tolist())

KeyError: 'vector'

## Prédictions

In [ ]:
# Faire des prédictions
y_pred_svc_tf = svc_model.predict(X_test_tf)
y_pred_nb_tf = nb_model.predict(X_test_tf)
y_pred_rf_tf = rf_model.predict(X_test_tf)

In [ ]:
# Faire des prédictions
y_pred_svc_w = svc_model_w.predict(X_test_w)
y_pred_nb_w = nb_model_w.predict(X_test_w)
y_pred_rf_w = rf_model_w.predict(X_test_w)


In [ ]:
y_pred_svc_b = svc_model_b.predict(X_test_b)
y_pred_nb_b = nb_model_b.predict(X_test_b)
y_pred_rf_b = rf_model_b.predict(X_test_b)

NameError: name 'svc_model_b' is not defined

### Métriques

In [ ]:
# Calculer les métriques
print("SVC:")
print(classification_report(y_test_b, y_pred_svc_b))

print("Naive Bayes:")
print(classification_report(y_test_b, y_pred_nb_b))

print("Random Forest:")
print(classification_report(y_test_b, y_pred_rf_b))

In [ ]:
# Calculer les métriques
print("SVC:")
print(classification_report(y_test, y_pred_svc_w))

print("Naive Bayes:")
print(classification_report(y_test, y_pred_nb_w))

print("Random Forest:")
print(classification_report(y_test, y_pred_rf_w))

In [ ]:
# Calculer les métriques
print("SVC:")
print(classification_report(y_test, y_pred_svc_tf))

print("Naive Bayes:")
print(classification_report(y_test, y_pred_nb_tf))

print("Random Forest:")
print(classification_report(y_test, y_pred_rf_tf))

### Les matrices de confusion :

In [ ]:
encoder = LabelEncoder()
encoder.fit(y_train)

def plot_confusion_matrix(y_test, y_pred, title):
    cm = confusion_matrix(y_test, y_pred)
    fig = px.imshow(cm, text_auto=True, labels=dict(y="True Label", x="Predicted Label"),
                    x=encoder.classes_, y=encoder.classes_, title=title)
    fig.show()

# Générer et visualiser les matrices de confusion
plot_confusion_matrix(y_test, y_pred_svc, "SVC Confusion Matrix")
plot_confusion_matrix(y_test, y_pred_nb, "Naive Bayes Confusion Matrix")
plot_confusion_matrix(y_test, y_pred_rf, "Random Forest Confusion Matrix")

In [ ]:
encoder = LabelEncoder()
encoder.fit(y_train)

def plot_confusion_matrix(y_test, y_pred, title):
    cm = confusion_matrix(y_test, y_pred)
    fig = px.imshow(cm, text_auto=True, labels=dict(y="True Label", x="Predicted Label"),
                    x=encoder.classes_, y=encoder.classes_, title=title)
    fig.show()

# Générer et visualiser les matrices de confusion
plot_confusion_matrix(y_test, y_pred_svc, "SVC Confusion Matrix")
plot_confusion_matrix(y_test, y_pred_nb, "Naive Bayes Confusion Matrix")
plot_confusion_matrix(y_test, y_pred_rf, "Random Forest Confusion Matrix")

In [ ]:
encoder = LabelEncoder()
encoder.fit(y_train)

def plot_confusion_matrix(y_test, y_pred, title):
    cm = confusion_matrix(y_test, y_pred)
    fig = px.imshow(cm, text_auto=True, labels=dict(y="True Label", x="Predicted Label"),
                    x=encoder.classes_, y=encoder.classes_, title=title)
    fig.show()

# Générer et visualiser les matrices de confusion
plot_confusion_matrix(y_test, y_pred_svc, "SVC Confusion Matrix")
plot_confusion_matrix(y_test, y_pred_nb, "Naive Bayes Confusion Matrix")
plot_confusion_matrix(y_test, y_pred_rf, "Random Forest Confusion Matrix")